In [41]:
import numpy as np
import torch
import torch.nn as nn
import os
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torchvision
import torchvision.transforms as transforms
from PIL import Image, ImageDraw
from numpy import genfromtxt
import skimage
print(os.listdir("facial-expression/fer2013/")) #список директорий по ссылке

['fer2013.csv']


In [42]:
# get the data
filname = 'facial-expression/fer2013/fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']
df=pd.read_csv('facial-expression/fer2013/fer2013.csv',names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emotion,pixels,usage
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [43]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y

In [44]:
X, Y = getData(filname)
num_class = len(set(Y))
print(num_class)

7


In [45]:
N, D = X.shape
X = X.reshape(N, 48, 48, 1)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train).long()
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test).long()

In [46]:
y_train[1]

tensor(4)

In [47]:
training_data = []
test_data = []
for i in range(len(X_train)):
    training_data.append([X_train[i], y_train[i]])
for i in range(len(X_test)):
    test_data.append([X_test[i], y_test[i]])

In [48]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.features1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size = (3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size = (3, 3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2) #все еще 64 канала, но каждый из слоев уменьшается в 2 раза
        )
        
        self.features2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = (3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size = (3, 3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2)
        )
        
        self.features3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size = (3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size = (3, 3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2)
        )
        
        self._to_linear = 6 * 6 * 256 #аналог falttten из Keras; перед подачей на полносвязный слой тензор
                                #нужно выпрямить; количество элементов = "количество слоев = 256" *
                                #"количество элементов в слое = (48 /2 / 2/ 2) ^ 2 (по сути нынешний размер пикчи
                                #после предыдущих пулингов и сверток)"
        self.features4 = nn.Sequential(
            nn.Linear(self._to_linear, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 7),
            nn.Softmax(dim = 1)
        )

    def forward(self, x):
        x = self.features1(x)
        x = self.features2(x)
        x = self.features3(x)
        x = x.view(-1, self._to_linear)
        x = self.features4(x)
        return x

net = Net()
print(net)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

Net(
  (features1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (features2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (features3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4)

In [50]:
PATH = 'model_filter.h5' # save model at this location after each epoch

EPOCHS = 2

batch_size = 100

for epochs in range(EPOCHS):
    
    for i in tqdm(range(0, len(X_train), batch_size)):
        
        batch_X = X_train[i:i + batch_size].view(-1, 1, 48, 48)
        
        batch_y = y_train[i:i + batch_size]
        
        net.zero_grad()
        
        outputs = net(batch_X)
        
        loss = loss_function(outputs, batch_y)
        
        loss.backward()
        
        optimizer.step()
            
    print(f"Epoch: {epochs}. Loss: {loss}")
torch.save(net.state_dict(), PATH)

  0%|          | 0/323 [00:00<?, ?it/s]

Epoch: 0. Loss: 1.6544814109802246


100%|██████████| 323/323 [15:38<00:00,  2.91s/it]

Epoch: 1. Loss: 1.606380581855774


In [51]:
real_class = torch.argmax(y_test[0])
i = 0
ans = net(X_test[i:i + batch_size].view(-1, 1, 48, 48))
for i in range(len(ans)):
    a = torch.argmax(ans[i])
    ans[i] = a
print(len(X_test), batch_size)

3589 100


In [102]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(0, len(X_train), batch_size)):
        real_class = y_train[i:i + batch_size]
        ans = net(X_train[i:i + batch_size].view(-1, 1, 48, 48))
        for i in range(len(ans)):
            ans[i] = torch.argmax(ans[i])
            real_class[i] = torch.argmax(real_class[i])
            if (real_class[i] == ans[i][0]):
                correct += 1
            total += 1
print("Accuracy on train: ", round(correct/total, 3))

100%|██████████| 323/323 [05:13<00:00,  1.03it/s]

Accuracy new:  0.713


In [53]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(0, len(X_test), batch_size)):
        real_class = y_test[i:i + batch_size]
        ans = net(X_test[i:i + batch_size].view(-1, 1, 48, 48))
        for i in range(len(ans)):
            ans[i] = torch.argmax(ans[i])
            real_class[i] = torch.argmax(real_class[i])
            if (real_class[i] == ans[i][0]):
                correct += 1
            total += 1
print("Accuracy on test: ", round(correct/total, 3))

100%|██████████| 36/36 [00:33<00:00,  1.09it/s]

Accuracy on test:  0.128


In [96]:
objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
y_pos = np.arange(len(objects))
print(y_pos)

[0 1 2 3 4 5 6]


In [97]:
def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
plt.show()

In [98]:
y_pred=net(X_test.view(-1, 1, 48, 48))
#print(y_pred)
y_test.shape

torch.Size([3589, 7])

In [99]:
y_pred[0]

tensor([1.5738e-03, 2.2008e-04, 9.6004e-04, 9.9250e-01, 1.7863e-03, 1.7977e-03,
        1.1657e-03], grad_fn=<SelectBackward>)

In [100]:
count = 0
for i in range(len(y_test)):
    max = torch.max(y_pred[i])
    for k in range(len(y_pred[i])):
        if (y_pred[i][k] == max):
            break
    for j in range(len(y_test[i])):
        if (y_test[i][j] == 1):
            break
    if (k == j):
        count += 1
        print(y_pred[i], y_test[i], max)
print(count/len(y_test))

tensor([0.0422, 0.0043, 0.1497, 0.0422, 0.1258, 0.1160, 0.5198],
       grad_fn=<SelectBackward>) tensor([2., 2., 2., 2., 2., 2., 2.]) tensor(0.5198, grad_fn=<MaxBackward1>)
tensor([1.2205e-02, 3.9959e-04, 2.5242e-03, 5.1493e-03, 7.4854e-03, 8.1290e-03,
        9.6411e-01], grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.9641, grad_fn=<MaxBackward1>)
tensor([0.0607, 0.0178, 0.0823, 0.2562, 0.0848, 0.0589, 0.4393],
       grad_fn=<SelectBackward>) tensor([2., 2., 2., 2., 2., 2., 2.]) tensor(0.4393, grad_fn=<MaxBackward1>)
tensor([5.7999e-04, 2.1212e-05, 2.0460e-03, 2.1679e-03, 6.4812e-03, 4.9076e-05,
        9.8865e-01], grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.9887, grad_fn=<MaxBackward1>)
tensor([0.2983, 0.0273, 0.0854, 0.0485, 0.0581, 0.0075, 0.4749],
       grad_fn=<SelectBackward>) tensor([0., 0., 0., 0., 0., 0., 0.]) tensor(0.4749, grad_fn=<MaxBackward1>)
tensor([0.1214, 0.0130, 0.0869, 0.0145, 0.2099, 0.0119, 0.5424],
       

tensor([0.0798, 0.0098, 0.0659, 0.0208, 0.3394, 0.0060, 0.4783],
       grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.4783, grad_fn=<MaxBackward1>)
tensor([0.1233, 0.0221, 0.0475, 0.0950, 0.1210, 0.0032, 0.5878],
       grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.5878, grad_fn=<MaxBackward1>)
tensor([0.0174, 0.0041, 0.0452, 0.0058, 0.3909, 0.0032, 0.5334],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.5334, grad_fn=<MaxBackward1>)
tensor([0.0388, 0.0023, 0.0695, 0.0100, 0.1409, 0.0022, 0.7362],
       grad_fn=<SelectBackward>) tensor([0., 0., 0., 0., 0., 0., 0.]) tensor(0.7362, grad_fn=<MaxBackward1>)
tensor([0.0169, 0.0010, 0.0081, 0.0090, 0.0188, 0.0048, 0.9414],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.9414, grad_fn=<MaxBackward1>)
tensor([0.0115, 0.0025, 0.0108, 0.0108, 0.0538, 0.0017, 0.9089],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 

tensor([0.1508, 0.0120, 0.2135, 0.0129, 0.1910, 0.1682, 0.2516],
       grad_fn=<SelectBackward>) tensor([5., 5., 5., 5., 5., 5., 5.]) tensor(0.2516, grad_fn=<MaxBackward1>)
tensor([0.1129, 0.0182, 0.0594, 0.0887, 0.1128, 0.0061, 0.6020],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.6020, grad_fn=<MaxBackward1>)
tensor([0.1095, 0.0095, 0.1248, 0.0022, 0.2204, 0.0135, 0.5201],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.5201, grad_fn=<MaxBackward1>)
tensor([1.3055e-03, 1.4509e-04, 5.5889e-03, 3.3194e-04, 1.8121e-02, 1.1284e-03,
        9.7338e-01], grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.9734, grad_fn=<MaxBackward1>)
tensor([0.1418, 0.0301, 0.0814, 0.2237, 0.1142, 0.0235, 0.3853],
       grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.3853, grad_fn=<MaxBackward1>)
tensor([7.9522e-03, 7.3420e-04, 1.6816e-02, 2.6418e-03, 5.9836e-02, 5.3361e-02,
        8.5866e-01], 

tensor([0.0269, 0.0031, 0.0666, 0.0009, 0.1581, 0.0456, 0.6987],
       grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.6987, grad_fn=<MaxBackward1>)
tensor([0.0852, 0.0054, 0.0758, 0.0036, 0.2731, 0.0009, 0.5560],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.5560, grad_fn=<MaxBackward1>)
tensor([0.1472, 0.0282, 0.1613, 0.0467, 0.1723, 0.0124, 0.4319],
       grad_fn=<SelectBackward>) tensor([2., 2., 2., 2., 2., 2., 2.]) tensor(0.4319, grad_fn=<MaxBackward1>)
tensor([0.3854, 0.0096, 0.0180, 0.0167, 0.1156, 0.0009, 0.4538],
       grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.4538, grad_fn=<MaxBackward1>)
tensor([0.0636, 0.0073, 0.0673, 0.1760, 0.2091, 0.0063, 0.4705],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.4705, grad_fn=<MaxBackward1>)
tensor([0.1557, 0.0340, 0.1093, 0.0272, 0.2667, 0.0239, 0.3832],
       grad_fn=<SelectBackward>) tensor([0., 0., 0., 0., 0., 0., 

tensor([0.0126, 0.0034, 0.0357, 0.0118, 0.2256, 0.0007, 0.7102],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.7102, grad_fn=<MaxBackward1>)
tensor([0.1076, 0.0671, 0.1547, 0.0196, 0.2493, 0.0465, 0.3552],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.3552, grad_fn=<MaxBackward1>)
tensor([0.0653, 0.0061, 0.1068, 0.0051, 0.3357, 0.0071, 0.4739],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.4739, grad_fn=<MaxBackward1>)
tensor([0.0248, 0.0097, 0.0399, 0.1427, 0.0575, 0.0183, 0.7070],
       grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.7070, grad_fn=<MaxBackward1>)
tensor([0.0158, 0.0021, 0.0222, 0.0392, 0.0938, 0.0010, 0.8259],
       grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.8259, grad_fn=<MaxBackward1>)
tensor([1.0113e-02, 5.2533e-04, 3.4254e-03, 8.4279e-03, 1.0584e-02, 1.4402e-03,
        9.6548e-01], grad_fn=<SelectBackward>) ten

tensor([0.0533, 0.0058, 0.0125, 0.1780, 0.0298, 0.0010, 0.7196],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.7196, grad_fn=<MaxBackward1>)
tensor([1.4454e-03, 8.5031e-05, 7.1543e-04, 7.2633e-02, 1.8903e-03, 2.5658e-03,
        9.2066e-01], grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.9207, grad_fn=<MaxBackward1>)
tensor([0.2345, 0.0334, 0.0762, 0.0103, 0.2992, 0.0046, 0.3419],
       grad_fn=<SelectBackward>) tensor([2., 2., 2., 2., 2., 2., 2.]) tensor(0.3419, grad_fn=<MaxBackward1>)
tensor([0.1181, 0.0058, 0.0691, 0.0063, 0.2857, 0.0020, 0.5131],
       grad_fn=<SelectBackward>) tensor([0., 0., 0., 0., 0., 0., 0.]) tensor(0.5131, grad_fn=<MaxBackward1>)
tensor([2.2815e-02, 8.4307e-04, 3.7188e-02, 7.8983e-03, 4.1292e-02, 2.1956e-03,
        8.8777e-01], grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.8878, grad_fn=<MaxBackward1>)
tensor([0.0920, 0.0281, 0.0752, 0.0667, 0.1775, 0.1000, 0.4604],
       

tensor([0.1273, 0.0153, 0.0533, 0.0401, 0.1417, 0.0053, 0.6169],
       grad_fn=<SelectBackward>) tensor([0., 0., 0., 0., 0., 0., 0.]) tensor(0.6169, grad_fn=<MaxBackward1>)
tensor([0.0459, 0.0023, 0.0570, 0.0051, 0.1704, 0.0194, 0.6998],
       grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.6998, grad_fn=<MaxBackward1>)
tensor([0.1210, 0.0125, 0.0856, 0.0055, 0.2700, 0.0043, 0.5012],
       grad_fn=<SelectBackward>) tensor([0., 0., 0., 0., 0., 0., 0.]) tensor(0.5012, grad_fn=<MaxBackward1>)
tensor([8.5199e-04, 1.1393e-04, 3.8870e-04, 4.9728e-02, 6.1081e-04, 3.7886e-04,
        9.4793e-01], grad_fn=<SelectBackward>) tensor([4., 4., 4., 4., 4., 4., 4.]) tensor(0.9479, grad_fn=<MaxBackward1>)
tensor([0.0627, 0.0242, 0.0906, 0.2424, 0.1273, 0.0453, 0.4076],
       grad_fn=<SelectBackward>) tensor([6., 6., 6., 6., 6., 6., 6.]) tensor(0.4076, grad_fn=<MaxBackward1>)
tensor([7.9649e-02, 8.4148e-03, 1.1290e-02, 8.8422e-03, 3.3533e-02, 4.9790e-04,
        8.5777e-01], 

In [101]:
from skimage import io
img = image.load_img('bek.jpg', grayscale=True, target_size=(48, 48))
show_img=image.load_img('bek.jpg', grayscale=False, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = model.predict(x)
#print(custom[0])
emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([48, 48]);

plt.gray()
plt.imshow(show_img)
plt.show()

m=0.000000000000000000001
a=custom[0]
for i in range(0,len(a)):
    if a[i]>m:
        m=a[i]
        ind=i
        
print('Expression Prediction:',objects[ind])

NameError: name 'image' is not defined

In [ ]:
from skimage import io
img = image.load_img('../input/testimages/wallpaper2you_443897.jpg', grayscale=True, target_size=(48, 48))
show_img=image.load_img('../input/testimages/wallpaper2you_443897.jpg', grayscale=False, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = model.predict(x)
#print(custom[0])
emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([48, 48]);

plt.gray()
plt.imshow(show_img)
plt.show()

m=0.000000000000000000001
a=custom[0]
for i in range(0,len(a)):
    if a[i]>m:
        m=a[i]
        ind=i
        
print('Expression Prediction:',objects[ind])